<a href="https://colab.research.google.com/github/emguzzi/MasterThesisDemo/blob/main/SpeechCommands/TruncatedSignatureTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install signatory
!git clone https://github.com/emguzzi/MasterThesisDemo.git
import sys
sys.path.append('./MasterThesisDemo/PenDigits/')
from RandomSignature import *
import signatory
import torch
from sklearn.ensemble import RandomForestClassifier
from google.colab import drive
drive.mount("/content/drive")
import pickle
import numpy as np
from tqdm.auto import tqdm
from scipy.special import expit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'MasterThesisDemo' already exists and is not an empty directory.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## load the preprocessed paths
## data has to be stored on drive in a folder called SpeechCommands

with open('/content/drive/MyDrive/SpeechCommands/paths_time.pkl','rb') as f:
    paths = pickle.load(f)
with open('/content/drive/MyDrive/SpeechCommands/y_train.pkl','rb') as f:
    y_train = pickle.load(f)
with open('/content/drive/MyDrive/SpeechCommands/y_test.pkl','rb') as f:
    y_test = pickle.load(f)
with open('/content/drive/MyDrive/SpeechCommands/y_validation.pkl','rb') as f:
    y_validation = pickle.load(f)
    
paths_torch = torch.tensor(paths)
paths_np = np.array(paths)

In [ ]:
truncated_sig = signatory.signature(paths_torch,3)

#remove terms of truncated signature of level 1
truncated_sig_subsampled = truncated_sig[:,132:]

#subsample the features
#omit_rate = 0.95
#ind = np.random.choice(range(1331),replace = False,size = int(1331*(1-omit_rate)))
#truncated_sig_subsampled = truncated_sig_subsampled[:,ind]

#train and test
truncated_sig_train = truncated_sig[:27864,:] 
truncated_sig_test = truncated_sig[27864:31639,:]

truncated_sig_train_subsampled = truncated_sig_subsampled[:27864,:] 
truncated_sig_test_subsampled = truncated_sig_subsampled[27864:31639,:]


In [ ]:
clf = RandomForestClassifier(n_estimators = 1000, max_depth = 100)

#clf.fit(truncated_sig_train,y_train)
#truncated_pred = clf.predict(truncated_sig_test)

clf.fit(truncated_sig_train_subsampled,y_train)
truncated_pred_subsampled = clf.predict(truncated_sig_test_subsampled)

#print('Accurcay for the truncated signature: ')
#print(np.mean(truncated_pred == y_test))
print('Accurcay for the subsampled truncated signature: ')
print(np.mean(truncated_pred_subsampled == y_test))


Accurcay for the subsampled truncated signature: 
0.7298013245033113


In [ ]:
## consider a mix of the features of the truncated signature and the randomized signature
## compute randomized and truncated signature
def sigmoid(x):
    return 1/(1+np.exp(-x))
def sigmoid2(x):
  return expit(x)

hparams = {
'varA':0.001, # best varA around 0.0005 - 0.001
'mean':0,
'res_size':300, # 132 for n= 2, 1464 for n = 3
'activation': sigmoid2
}
# generate vector fields
[As,bs] = get_random_coeff(paths_np.shape[2],hparams)
# compute signature
rand_sig = compute_signature_vect(As,bs,paths_np,hparams)

truncated_sig = signatory.signature(paths_torch,2).numpy()

joined_features = np.hstack([truncated_sig,rand_sig])
joined_features_train = joined_features[:27864,:] 
joined_features_test = joined_features[27864:31639,:]
# subsample the truncated signature and then add elements of the randomized signature
# to keep the number of features constant
truncated_sig_rate = 0.3
ind = np.random.choice(range(300),replace = False,size = int(300*truncated_sig_rate))
truncated_sig_subsampled = truncated_sig[:,ind]

number_rsig = 300 - int(300*(truncated_sig_rate))
randomized_sig_subsampled = rand_sig[:,:number_rsig]
features = np.hstack([truncated_sig_subsampled,randomized_sig_subsampled])

## train and evaluate
features_train = features[:27864,:] 
features_test = features[27864:31639,:]

clf = RandomForestClassifier(n_estimators = 1000, max_depth = 100)
#clf.fit(features_train,y_train)
#merged_pred = clf.predict(features_test)

clf.fit(joined_features_train,y_train)
joined_pred = clf.predict(joined_features_test)

print('Accurcay for the subsampled truncated signature: ')
#print(np.mean(merged_pred == y_test))
print('Accuracy for joined truncated and randomized signature:')
print(np.mean(joined_pred == y_test))


  0%|          | 0/320 [00:00<?, ?it/s]

IndexError: ignored